In [11]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, concat, col, regexp_extract,  regexp_replace, coalesce, lit, date_format

In [2]:
spark = SparkSession.builder \
    .appName("Notebook Glue Job Simulation") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/02/01 11:24:11 WARN Utils: Your hostname, 01120230807W11I resolves to a loopback address: 127.0.1.1; using 172.24.33.137 instead (on interface eth0)
25/02/01 11:24:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/01 11:24:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
json_path = "../data/amadeus_info_example.json"
df = spark.read.option("multiline", "true").json(json_path)


In [4]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- instantTicketingRequired: boolean (nullable = true)
 |    |    |-- isUpsellOffer: boolean (nullable = true)
 |    |    |-- itineraries: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- duration: string (nullable = true)
 |    |    |    |    |-- segments: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- aircraft: struct (nullable = true)
 |    |    |    |    |    |    |    |-- code: string (nullable = true)
 |    |    |    |    |    |    |-- arrival: struct (nullable = true)
 |    |    |    |    |    |    |    |-- at: string (nullable = true)
 |    |    |    |    |    |    |    |-- iataCode: string (nullable = true)
 |    |    |    |    |    |    |-- blacklistedInEU: boolean (nullable = true)
 |    |    |    

In [14]:
exploded_df = df.withColumn("flight_offer", explode("data"))

# Extrae los datos y mantén las columnas necesarias
flights_df = exploded_df.selectExpr(
    "flight_offer.id as flight_id",
    "flight_offer.itineraries[0].segments[0].departure.iataCode as departure_airport",
    "flight_offer.itineraries[0].segments[0].departure.at as departure_time",
    "flight_offer.itineraries[0].segments[0].departure.terminal as departure_terminal",
    "flight_offer.itineraries[0].segments[0].arrival.iataCode as arrival_airport",
    "flight_offer.itineraries[0].segments[0].arrival.at as arrival_time",
    "flight_offer.itineraries[0].segments[0].carrierCode as carrier",
    "flight_offer.itineraries[0].segments[0].number as flight_number",
    "flight_offer.itineraries[0].segments[0].aircraft.code as aircraft_code",
    "flight_offer.itineraries[0].segments[0].operating.carrierCode as operating_company_code",
    "flight_offer.itineraries[0].segments[0].duration as flight_duration",
    "flight_offer.itineraries[0].segments[0].numberOfStops as number_of_stops",
    "flight_offer.itineraries[0].segments[0].blacklistedInEU as blacklisted_in_eu",
    "flight_offer.price.total as total_price",
    "flight_offer.price.currency as currency"
)

In [15]:
flights_df.show()

+---------+-----------------+-------------------+------------------+---------------+-------------------+-------+-------------+-------------+----------------------+---------------+---------------+-----------------+-----------+--------+
|flight_id|departure_airport|     departure_time|departure_terminal|arrival_airport|       arrival_time|carrier|flight_number|aircraft_code|operating_company_code|flight_duration|number_of_stops|blacklisted_in_eu|total_price|currency|
+---------+-----------------+-------------------+------------------+---------------+-------------------+-------+-------------+-------------+----------------------+---------------+---------------+-----------------+-----------+--------+
|        1|              MAD|2024-09-11T07:05:00|                 2|            AMS|2024-09-11T09:40:00|     UX|         1091|          789|                    UX|        PT2H35M|              0|            false|      62.80|     EUR|
|        2|              MAD|2024-09-11T15:00:00|           

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `ori` cannot be resolved. Did you mean one of the following? [`flight_id`, `departure_airport`, `departure_time`, `departure_terminal`, `arrival_airport`, `arrival_time`, `carrier`, `flight_number`, `aircraft_code`, `company_code`, `flight_duration`, `number_of_stops`, `blacklisted_in_eu`, `total_price`, `currency`].

In [23]:
from pyspark.sql.functions import col, explode

# Seleccionar columnas relevantes del itinerario
details_df = itineraries_df.select(
    "ori", "dest", "year", "month", "week", "day",
    col("itinerary.price.raw").alias("price_raw"),
    col("itinerary.legs").alias("legs"),
    col("itinerary.farePolicy").alias("farePolicy"),
    col("itinerary.isSelfTransfer").alias("isSelfTransfer"),
    col("itinerary.isProtectedSelfTransfer").alias("isProtectedSelfTransfer")
)

# Explotar legs para dividir los vuelos en filas individuales
legs_df = details_df.withColumn("leg", explode("legs")) \
    .withColumn("marketing", explode("leg.carriers.marketing")) \
    .withColumn("segments", explode("leg.segments"))

# Definir columnas finales de salida
selected_columns = [
    "ori", "dest", "month", "week", "day", "price_raw",
    col("leg.id").alias("leg_id"),
    col("leg.origin.name").alias("origin_name"),
    col("leg.origin.entityId").alias("origin_entityId"),
    col("leg.origin.displayCode").alias("origin_airport"),
    col("leg.origin.city").alias("origin_city"),
    col("leg.origin.country").alias("origin_country"),
    col("leg.destination.name").alias("destination_name"),
    col("leg.destination.entityId").alias("destination_entityId"),
    col("leg.destination.displayCode").alias("destination_airport"),
    col("leg.destination.city").alias("destination_city"),
    col("leg.destination.country").alias("destination_country"),
    col("leg.durationInMinutes").alias("duration_in_minutes"),
    col("leg.isSmallestStops").alias("is_smallest_stops"),
    col("leg.timeDeltaInDays").alias("time_delta_days"),
    col("leg.departure").alias("departure_time"),
    col("leg.arrival").alias("arrival_time"),
    col("leg.carriers.operationType").alias("company_operation_type"),
    col("segments.flightNumber").alias("flight_number"),
    col("segments.marketingCarrier.name").alias("marketing_company_name"),
    col("segments.marketingCarrier.alternateId").alias("marketing_company_code"),
    col("segments.operatingCarrier.name").alias("operating_company_name"),
    col("segments.operatingCarrier.alternateId").alias("operating_company_code"),
    col("isSelfTransfer"),
    col("isProtectedSelfTransfer"),
    col("farePolicy.isChangeAllowed"),
    col("farePolicy.isPartiallyChangeable"),
    col("farePolicy.isCancellationAllowed"),
    col("farePolicy.isPartiallyRefundable")
]

# Crear el DataFrame final con los datos limpios y estructurados
final_legs_df = legs_df.select(*selected_columns)

# Guardar en formato parquet en S3 (descomentar si es necesario)
# output_flights_path = "s3://my-aws-project-v1/transformed/skyscanner/flights"
# final_legs_df.write.format("parquet").mode("overwrite").partitionBy("ori", "dest", "year", "month", "week", "day").save(output_flights_path)


In [24]:
final_legs_df.show()

+---+----+-----+----+---+---------+--------------------+-----------+--------------+-----------+--------------+------------------+-------------------+----------------+-------------------+-------------------+-----------------+---------------+-------------------+-------------------+----------------------+-------------+----------------------+----------------------+----------------------+----------------------+--------------+-----------------------+---------------+---------------------+---------------------+---------------------+
|ori|dest|month|week|day|price_raw|              leg_id|origin_name|origin_airport|origin_city|origin_country|  destination_name|destination_airport|destination_city|destination_country|duration_in_minutes|is_smallest_stops|time_delta_days|     departure_time|       arrival_time|company_operation_type|flight_number|marketing_company_name|marketing_company_code|operating_company_name|operating_company_code|isSelfTransfer|isProtectedSelfTransfer|isChangeAllowed|isPar

In [25]:
spark.stop()